## 使用SQL进行LLaMA2聊天

开源的本地LLMs非常适合考虑用于需要数据隐私的任何应用程序。

SQL是一个很好的例子。

本手册展示了如何使用各种本地版本的LLaMA2在本地运行进行文本到SQL的转换。

## 软件包

In [ ]:
# 导入所需的库
! pip install langchain replicate

## LLM

有几种访问 LLaMA2 的方式。

要在本地运行，我们使用 Ollama.ai。

有关安装和设置的详细信息，请参见[这里](/docs/integrations/chat/ollama)。

此外，我们还有关于本地 LLMs 的完整指南，请参见[这里](/docs/guides/development/local_llms)。

要使用一个非私有的外部 API，我们可以使用 Replicate。

In [1]:
# 本地
from langchain_community.chat_models import ChatOllama

# 创建一个名为llama2_chat的ChatOllama对象，使用llama2:13b-chat模型
llama2_chat = ChatOllama(model="llama2:13b-chat")

# 创建一个名为llama2_code的ChatOllama对象，使用codellama:7b-instruct模型
llama2_code = ChatOllama(model="codellama:7b-instruct")

# API
from langchain_community.llms import Replicate

# 设置REPLICATE_API_TOKEN
# REPLICATE_API_TOKEN = getpass()
# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

# 设置replicate_id
replicate_id = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

# 创建一个名为llama2_chat_replicate的Replicate对象，使用指定的模型和输入参数
llama2_chat_replicate = Replicate(
    model=replicate_id, input={"temperature": 0.01, "max_length": 500, "top_p": 1}
)

Init param `input` is deprecated, please use `model_kwargs` instead.


In [2]:
# 简单地设置我们想要使用的LLM
llm = llama2_chat

## 数据库

连接到一个SQLite数据库。

要创建这个特定的数据库，您可以使用以下代码并按照[这里](https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/StructuredLlama.ipynb)显示的步骤进行操作。

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)
# 创建一个SQLDatabase对象，连接到名为"nba_roster.db"的SQLite数据库，设置sample_rows_in_table_info参数为0

def get_schema(_):
    return db.get_table_info()
# 定义一个名为get_schema的函数，接受一个参数_，返回db.get_table_info()的结果

def run_query(query):
    return db.run(query)
# 定义一个名为run_query的函数，接受一个参数query，返回db.run(query)的结果

## 查询 SQL 数据库

请按照此处的可运行工作流程[链接](https://python.langchain.com/docs/expression_language/cookbook/sql_db)进行操作。

In [4]:
# 提示
from langchain_core.prompts import ChatPromptTemplate

# 根据 SQL 数据库的类型（如 MySQL、Microsoft SQL Server 等）更新模板
template = """根据下面的表结构，编写一个 SQL 查询来回答用户的问题：
{schema}

问题：{question}
SQL 查询："""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "给定一个输入问题，将其转换为 SQL 查询。不需要前言。"),
        ("human", template),
    ]
)

# 链接到查询
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)  # 从 get_schema 函数中获取表结构
    | prompt
    | llm.bind(stop=["\nSQLResult:"])  # 绑定到 llm，停止标记为 "\nSQLResult:"
    | StrOutputParser()  # 使用 StrOutputParser 解析输出
)

sql_response.invoke({"question": "Klay Thompson 在哪个球队？"})  # 调用 sql_response，传入问题参数

' SELECT "Team" FROM nba_roster WHERE "NAME" = \'Klay Thompson\';'

我们可以查看结果：

* [LangSmith trace](https://smith.langchain.com/public/afa56a06-b4e2-469a-a60f-c1746e75e42b/r) LLaMA2-13 复制 API
* [LangSmith trace](https://smith.langchain.com/public/2d4ecc72-6b8f-4523-8f0b-ea95c6b54a1d/r) LLaMA2-13 本地

In [15]:
# 链式回答
template = """根据下面的表模式、问题、SQL查询和SQL响应，编写一个自然语言回答：
{schema}

问题: {question}
SQL查询: {query}
SQL响应: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "给定一个输入问题和SQL响应，将其转换为自然语言回答。没有前言。",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response)  # 将sql_response赋值给query
    | RunnablePassthrough.assign(
        schema=get_schema,  # 获取表模式
        response=lambda x: db.run(x["query"]),  # 运行SQL查询并将结果赋值给response
    )
    | prompt_response  # 使用prompt_response生成自然语言回答
    | llm  # 使用llm模型生成最终回答
)

full_chain.invoke({"question": "有多少个唯一的团队？"})  # 调用full_chain并传入问题参数

AIMessage(content=' Based on the table schema and SQL query, there are 30 unique teams in the NBA.')

我们可以查看结果：

* [LangSmith跟踪](https://smith.langchain.com/public/10420721-746a-4806-8ecf-d6dc6399d739/r) LLaMA2-13复制API
* [LangSmith跟踪](https://smith.langchain.com/public/5265ebab-0a22-4f37-936b-3300f2dfa1c1/r) LLaMA2-13本地

## 与 SQL 数据库交谈

接下来，我们可以添加内存。

In [7]:
# 导入必要的模块
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 定义模板字符串，用于生成对话提示
template = """给定一个输入问题，将其转换为 SQL 查询。不需要序文。基于下面的表结构，编写一个 SQL 查询，以回答用户的问题：
{schema}
"""
# 创建对话提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 创建对话缓存内存
memory = ConversationBufferMemory(return_messages=True)

# 链式调用，将查询与内存连接起来
from langchain_core.runnables import RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# 定义保存函数
def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]

# 执行 SQL 查询并保存结果到内存
sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

' SELECT "Team" FROM nba_roster WHERE "NAME" = \'Klay Thompson\';'

In [21]:
# Chain to answer
# 定义模板字符串，包含表格模式、问题、SQL查询和SQL响应，用于生成自然语言响应
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
# 从消息中创建聊天提示模板
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

# 创建完整的处理链
full_chain = (
    RunnablePassthrough.assign(query=sql_response_memory)  # 将SQL响应分配给query
    | RunnablePassthrough.assign(  # 分配schema和response
        schema=get_schema,  # 获取表格模式
        response=lambda x: db.run(x["query"]),  # 运行SQL查询并获取响应
    )
    | prompt_response  # 使用聊天提示模板生成自然语言响应
    | llm  # 使用语言模型生成最终的自然语言响应
)

# 调用完整的处理链并传入问题
full_chain.invoke({"question": "What is his salary?"})

AIMessage(content=' Sure! Here\'s the natural language response based on the given input:\n\n"Klay Thompson\'s salary is $43,219,440."')

这是[追踪链接](https://smith.langchain.com/public/54794d18-2337-4ce2-8b9f-3d8a2df89e51/r)。